In [288]:
from joblib import Parallel, delayed

def generate_row(df):
    df = df.copy()
    df["src"] = "output"
    df["time"] = (df["time"].max()-df["time"]).dt.total_seconds()
    df["x"] = df["x"] - df["x"].iloc[0]
    df["y"] = df["y"] - df["y"].iloc[0]
    df.iloc[:-1,-1] = "input_" + pd.Series(np.arange(df.shape[0]-1)).astype(str)
    df=df.melt(id_vars="src")
    df["colname"] = df.src + "_" + df.variable
    df=df.drop(columns=["src", "variable"]).set_index("colname")
    df=df.transpose().reset_index(drop=True)
    df.columns.name = None
    df=df.sort_index(axis=1).drop(["output_time"], axis=1)
    return df

def generate_model_frame(df, knowns):
    window_size = knowns + 1
    skipped_windows = 4

    windows = df.reset_index(drop=True).groupby('track').rolling(window_size)
    windows = (win for i, win in enumerate(windows) if len(win) == window_size and i % (skipped_windows + 1) == 0)
    windows = list(windows)
    res = Parallel(n_jobs=-1)(delayed(generate_row)(win) for win in tqdm(windows))
    #res = (generate_row(win) for win in tqdm(windows))
    return pd.concat(res, ignore_index=True)


In [ ]:
from pathlib import Path
targetrows_knowns = 3, 6, 9, 12
for target in targetrows_knowns
    path = f"../workdir/AIS-ModelFrames/aisdk-2023-11-08-xs_2_kkn_train_{target}.csv":
    if not path.exists(path):         n_model = generate_model_frame(train, target)
           t_model = generate_model_frame(test, target)
          ain_model.to_csv(f"index=False)
         est_model.to_csv(f"../workdir/AIS-ModelFrames/aisdk-2023-11-08-xs_2_kkn_test_{target}.csv", index=False)
        print(f"Done with {target}")
    

## Resample Idea (nicht weiter verfolgt)

In [113]:
time_resample = "10s"
resampled = example.resample(time_resample, level="time").mean().interpolate(method="time")
resampled

In [114]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(example['x'], example['y'], label='Example')
plt.plot(resampled['x'], resampled['y'], label=f'Resampled {time_resample}')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('X and Y Coordinates Through Time')
plt.legend()
plt.show()

In [112]:
example

In [124]:
X_resampled = X.reset_index().drop(columns=["epoch_time", "degree", "distance", "timepoint"]).set_index(['track','time','mmsi'])
X_resampled = X_resampled.groupby(["track","mmsi"]).apply(lambda df: df.resample(time_resample, level="time").mean().interpolate(method="time"))
X_resampled.head()

In [126]:
import gc
gc.collect()

In [127]:
X_resampled.groupby("track").size().describe()

Ziel: Bei Zeit 0

Beispiele für 3:
1. -5s, -10s, -15s:
2. -5s, -10s, -20s:
3. -30s, -60s, -61s:

In [ ]:
target_y, target_x               , old5s_x, old5s_y, old10s_x, old10s_y

In [ ]:
target_y, target_x,           old5s_x, old5s_y, old10s_x, old10s_y, old15s_x, old15s_y, old20s_x, old20s_y,

In [ ]:
9 Eingänge
8 hidden Neuron
2 Ausgänge


In [278]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm.auto import tqdm

In [279]:
# https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
data_in = pd.read_csv("../workdir/AIS-KNN-Files/aisdk-2023-11-08-xs_2_kkn.csv", lineterminator="$", header=None)

# KKN Data Analysis

In [280]:
# (bewegung, zeitpunkte, daten)
resshaped = data_in.values.reshape((-1, 100, 6))
# resshaped[0,0,:]
# X,Y and Degree,Distance are the same only different coordinate systems
# MMSI,Time,X,Y,Degree,Distance

In [281]:
resshaped.shape

In [282]:
resshaped.shape

In [283]:
tracks = range(resshaped.shape[0])
timepoints = range(resshaped.shape[1])
columns = range(resshaped.shape[2])

# stolen from https://github.com/sktime/sktime/blob/v0.11.4/sktime/datatypes/_panel/_convert.py#L608-L668
index = pd.MultiIndex.from_product([tracks, timepoints, columns], names=['track', 'timepoint', 'column'])
X = pd.Series(resshaped.flatten(), index=index)
X = X.unstack(level='column')
X.columns = ['mmsi', 'epoch_time', 'x', 'y', 'degree', 'distance']
X['time'] = pd.to_datetime(X['epoch_time'], unit='s')

In [284]:
# Time Ranges
X.groupby('track')["time"].diff().dt.total_seconds().describe()

In [285]:
secs = X.groupby('track')["time"].diff().dt.total_seconds()
_ = sns.histplot(secs, binwidth=1)

In [286]:
example = X.query("track == 0").copy()
example.time = example.time - example.time.min()
example = example.reset_index().set_index(['track','time','mmsi']).drop(columns=["epoch_time", "degree", "distance", "timepoint"])

1. 3 Bekannte, 1 Ziel
2. 6 Unbekannte, 1 Ziel
3. 12 Unbekannte, 1 Ziel

In [287]:
train_ships = pd.Series(X.mmsi.unique()).sample(frac=0.7, random_state=42)
tosplit = X.reset_index().drop(columns=["epoch_time", "degree", "distance", "timepoint"])
train = tosplit.loc[tosplit.mmsi.isin(train_ships),:].drop(columns=["mmsi"])
test = tosplit.loc[~tosplit.mmsi.isin(train_ships),:].drop(columns=["mmsi"])
train.shape[0]/tosplit.shape[0]
